In [1]:
import numpy as np
import pandas as pd
import requests
import geopandas as gpd
from shapely import wkt
from shapely.geometry import MultiPoint

### API Calls

        Census — Zip Codes Business Patterns: https://www.census.gov/data/developers/data-sets/cbp-zbp/zbp-api.2018.html#list-tab-353702932
                2013 variables: https://api.census.gov/data/2013/zbp/variables.html
                2018 variables: https://api.census.gov/data/2018/zbp/variables.html

In [2]:
years = [2013,2018]

# dictionary to store separate year df
zbp_dict = {}

for year in years:
    url = f'https://api.census.gov/data/{year}/zbp'

    params = {
        'get': 'EMP,ESTAB',     # employee count and establishment count
        'for': 'zipcode:*'
    }

    response = requests.get(url, params = params)

    if response.status_code == 200:
        data = response.json()
        df = pd.DataFrame(data[1:], columns = data[0])

            # fix column headers
        df.columns = df.columns.str.lower()
        df = df.rename(columns = {'emp':'employee_count','estab':'business_count','zip code':'zip_code'})
        
            # filter zichangeodes → 10s & 11s grabs NYC + lower tier → https://simple.wikipedia.org/wiki/List_of_ZIP_Code_prefixes
        df = df[df['zip_code'].str.startswith(('10', '11'))].reset_index(drop = True)
        zbp_dict[year] = df
    else:
        print(f"Error for {year}: {response.status_code}, {response.text}")

zbp13_df = zbp_dict[2013]
zbp18_df = zbp_dict[2018]

        Census — American Community Survey 5-Year: https://www.census.gov/data/developers/data-sets/acs-5year.2018.html#list-tab-1806015614
                2013 variables: https://api.census.gov/data/2013/acs/acs5/variables.html
                2018 variables: https://api.census.gov/data/2018/acs/acs5/variables.html

In [3]:
# dictionary to store separate year df
acs_dict = {}

for year in years:
    url = f'https://api.census.gov/data/{year}/acs/acs5'

    params = {
        'get':'B01003_001E',                   # total population estimate
        'for':'zip code tabulation area:*',    # ZCTAs not ZIPs
        'in':'state:36'                        # 36 = new york 
    }

    response = requests.get(url, params = params)

    if response.status_code == 200:
        data = response.json()
        df = pd.DataFrame(data[1:], columns = data[0])
        
            # fix column headers
        df.columns = df.columns.str.lower()
        df = df.rename(columns = {'b01003_001e':'population','zip code tabulation area':'zcta'})
        acs_dict[year] = df
    else:
        print(f'Error for {year}: {response.status_code}, {response.text}')

acs13_df = acs_dict[2013]
acs18_df = acs_dict[2018]

        Employment data uses ZIP and population data uses ZCTA so we need to use a crosswalk to merge

        HRSA ZIP to ZCTA Crosswalk: https://data.hrsa.gov/DataDownload/GeoCareNavigator/ZIP%20Code%20to%20ZCTA%20Crosswalk.xlsx


In [4]:
cross_df = pd.read_excel('ZIP Code to ZCTA Crosswalk.xlsx')
cross_df.columns = cross_df.columns.str.lower()
cross_df = cross_df[cross_df['state'] == 'NY']
cross_df['zcta'] = cross_df['zcta'].astype(int)
cross_df = cross_df[['zip_code','zcta','zip_join_type']].reset_index(drop = True)

### Merging

In [5]:
# merge to ZBP to get one row per ZIP
zbp_merged = pd.merge(zbp18_df, zbp13_df, on = 'zip_code', how = 'left')
zbp_merged = zbp_merged.rename(columns = {'employee_count_x':'employee_count_2018','business_count_x':'business_count_2018','employee_count_y':'employee_count_2013','business_count_y':'business_count_2013'})
zbp_merged = zbp_merged[['zip_code','employee_count_2013','employee_count_2018','business_count_2013','business_count_2018']].astype(int)

# merge to ACS to get one row per ZCTA
acs_merged = pd.merge(acs18_df, acs13_df, on = 'zcta', how = 'left')
acs_merged = acs_merged.rename(columns = {'population_x':'population_2018','population_y':'population_2013'})

In [6]:
# use crosswalk to convert ZCTA to ZIP
acs_merged['zcta'] = acs_merged['zcta'].astype(int)
acs_merged = pd.merge(acs_merged, cross_df, on = 'zcta', how = 'left')
acs_merged = acs_merged[acs_merged['zip_code'].astype(str).str.startswith(('10', '11'))].reset_index(drop = True)

# some duplicated ZIPs → keep 'ZIP matches ZCTA' but if that isn't true for a zip, then use 'Spaital join to ZCTA'"ZIP Code to ZCTA Crosswalk.xlsx"
acs_merged['priority'] = (acs_merged['zip_join_type'] == 'Zip matches ZCTA').astype(int)
acs_merged = acs_merged.sort_values(by = ['zip_code','priority'], ascending = [True, False]).drop_duplicates(subset = 'zip_code', keep = 'first').reset_index(drop = True)
acs_merged = acs_merged[['zip_code','population_2013','population_2018']].astype(int)

In [7]:
# merge ZBP and ACS
zip_df = pd.merge(zbp_merged, acs_merged, on = 'zip_code', how = 'left')
zip_df = zip_df.dropna().reset_index(drop = True)     # 4 rows did not merge with population data → drop

### BigQuery public data

        Zip Code Geometry
        Subway Station Ridership and Geometry

In [8]:
zip_geo_df = pd.read_csv('NYC zip geometry.csv')
sub_df = pd.read_csv('subway ridership.csv')
#separate geometry and boroughs
sub_geo = sub_df[['station','borough','station_geom']]

In [9]:
# ZIP level population and employment still contains ZIPs that are not in NYC
# geometry data contains only NYC, excluding Staten Island
nyc_df = pd.merge(zip_geo_df, zip_df, on = 'zip_code', how = 'left')
nyc_df = nyc_df.dropna().reset_index(drop = True)       # 1 row did not merge with population data → drop

In [10]:
# subway data is partitioned by station and route → group by 
sub_df['route_count'] = sub_df['route'].str.split().apply(len)      # count the train lines in the row
sub_df = sub_df[['station','borough','route_count','ridership_2013','ridership_2018']]
sub_df = sub_df.groupby(['station','borough']).sum().reset_index()

In [11]:
# the geometry points are different for duplicate rows of the same station
# this probably means the points are something like entrences instead of the center point of the station → need to get center
sub_geo['station_geom'] = sub_geo['station_geom'].apply(wkt.loads)
sub_geo = gpd.GeoDataFrame(sub_geo, geometry = 'station_geom', crs = 'EPSG:4326')
sub_geo = sub_geo.groupby(['station', 'borough'], as_index = False)['station_geom'].agg(lambda s: MultiPoint(list(s)).centroid)

# merge back with ridership data
sub_df = pd.merge(sub_df, sub_geo, on = ['station', 'borough'], how = 'left')

In [12]:
# turn into geo dataframe
sub_gdf = gpd.GeoDataFrame(sub_df, geometry = 'station_geom', crs = 'EPSG:4326')
nyc_df['zip_code_geom'] = nyc_df['zip_code_geom'].apply(wkt.loads)
nyc_gdf = gpd.GeoDataFrame(nyc_df, geometry = 'zip_code_geom', crs = 'EPSG:4326')

### Spatial Buffering → what zip codes are within 0.5 miles of a station

In [13]:
# change crs for better accuracy → feet instead of degrees
sub_gdf = sub_gdf.to_crs(epsg = 2263)
nyc_gdf = nyc_gdf.to_crs(epsg = 2263)

# create a 0.5 mile buffer & set this as the active geometry column
sub_gdf['buffer_geom'] = sub_gdf.geometry.buffer(2640)
sub_gdf = sub_gdf.set_geometry('buffer_geom')

# only keep station, borough and the buffer geom
sub_gdf = sub_gdf[['station','borough','buffer_geom']]

# spatial join → get zip codes that intersect with the buffer
zips_near_station = gpd.sjoin(nyc_gdf, sub_gdf[['station','borough','buffer_geom']], how = 'inner', predicate = 'intersects')

In [14]:
# spatial join row multiplies → a row for each station to zip combination → groupby on stations
spatial_df = zips_near_station[['station','population_2013','population_2018','employee_count_2013','employee_count_2018','business_count_2013','business_count_2018']]
spatial_df = spatial_df.groupby('station').sum().reset_index()

# add back ridership data and add borough for control
spatial_df = pd.merge(spatial_df, sub_df, on = 'station', how = 'left')
#spatial_df = pd.merge(spatial_df, sub_boro, on = 'station', how = 'left')

In [15]:
# add growth / decline
spatial_df['rider_change'] = spatial_df['ridership_2018'] - spatial_df['ridership_2013']
spatial_df['pop_change'] = spatial_df['population_2018'] - spatial_df['population_2013']
spatial_df['emp_change'] = spatial_df['employee_count_2018'] - spatial_df['employee_count_2013']
spatial_df['bus_change'] = spatial_df['business_count_2018'] - spatial_df['business_count_2013']

### Save off dataframes

In [ ]:
# dataset for regression
spatial_df.to_csv('regression data.csv', index = False)

# zip codes that are within the buffer zone → for mapping
buffer_zip_list = zips_near_station['zip_code'].unique()
buffer_zip_df = nyc_df[nyc_df['zip_code'].isin(buffer_zip_list)]
buffer_zip_df.to_csv('zip codes close to subways.csv', index = False)